# Analysis of earthquake density per tectonic region

In [ ]:
%matplotlib inline
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('/Users/mpagani/Projects/original/oq-engine/')
sys.path.append('/Users/mpagani/Projects/oq-man/')

from openquake.man.model import read
from openquake.man.single.areas import _get_area
from shapely.wkt import loads
from openquake.hazardlib.source.area import AreaSource

In [ ]:
modeln = './../../data/xml/EOS_SEAsia.xml'
sm = read(modeln)

In [ ]:
dat_as = {}
for src in sm:
    if isinstance(src, AreaSource):
        rtes = src.mfd.get_annual_occurrence_rates()
        mmin, mmax = src.mfd.get_min_max_mag()
        area = _get_area(loads(src.polygon.wkt))
        dat_as[src.source_id] = {'mmax': mmax, 'area': area, 'rtes': rtes, 'src': src}

In [ ]:
def plot_eqk_density_hist(srcs):
    """
    """
    dens = []
    ares = []
    dtrt = {}
    srctr = {}

    for src in srcs:

        if isinstance(src, AreaSource):
            trte = sum([rte[1] for rte in src.mfd.get_annual_occurrence_rates()])
            area = _get_area(loads(src.polygon.wkt))
            dens.append(trte/area)
            trt = src.tectonic_region_type
            if trt in dtrt:
                dtrt[trt].append(trte/area) 
                srctr[trt] += 1
            else: 
                dtrt[trt] = [trte/area]
                srctr[trt] = 1

    bins = np.logspace(-8, -3, num=20)
    hist, bin_edges = np.histogram(dens, bins=bins)
    #hist, bin_edges = np.histogram(dens)
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(1, 1, 1)
    plt.hist(dens, bins=bin_edges) 
    plt.xlabel(r'Seismicity density, [eqks/(yr*sqkm)]')
    plt.grid(which='both', linestyle='--')
    plt.xscale('log')
    plt.title('All')

    for trt in sorted(dtrt.keys()):
        hist, bin_edges = np.histogram(dtrt[trt], bins=bin_edges)
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(1, 1, 1)
        plt.hist(dtrt[trt], bins=bin_edges) 
        plt.xlabel(r'Seismicity density, [eqks/(yr*sqkm)]')
        plt.grid(which='both', linestyle='--')
        plt.xscale('log')
        plt.title(trt)
        print '{0:20s} {1:4d} {2:9.4e} {3:9.4e}'.format(trt, sum(hist), min(dtrt[trt]), max(dtrt[trt]))

    for key in srctr:
        print (key, srctr[key])

In [ ]:
plot_eqk_density_hist(sm)